In [27]:
import torch
import numpy as np
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [28]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model
using cached model
using cached model


In [37]:
df = pd.read_excel('당신이 가장 믿고 있는 자신의 능력은 무엇입니까.xlsx', index_col = None)
df

,num,temp_text,text
0,0,문제해결능력,문제해결(Problem Solving)은 일상적인 생활에서 발생하는 문제 상황을 가...
1,1,고집,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...
2,2,신념,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...
3,3,주장,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...
4,4,주관 뚜렷,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...
...,...,...,...
257,257,보통 이상은 한다,어떠한 성취 상황에서 성공적으로 수행할 수 있다는 자신의 능력에 대한 신념을 자기효...
258,258,보통은 한다,어떠한 성취 상황에서 성공적으로 수행할 수 있다는 자신의 능력에 대한 신념을 자기효...
259,259,길 찾는 것,공간 사이의 관계를 파악할 수 있는 능력을 공간지각능력이라고 해요. 입체적인 물체나...
260,260,다른 사람들을 편안하게 만드는거,당신은 친화성이 높은 것으로 보이네요. 심리학에서 개인의 성격을 설명하는 5요인 이...


In [38]:
vector_list = []
for i in df['temp_text'] :
    test = i
    toked = tok(test)
    input_ids = torch.tensor(vocab[toked]).unsqueeze(0)
    _, vector = bertmodel(input_ids = input_ids)
    vector_list.append( (vector.detach().numpy())/len(toked))

df['vector'] = vector_list
df

,num,temp_text,text,vector
0,0,문제해결능력,문제해결(Problem Solving)은 일상적인 생활에서 발생하는 문제 상황을 가...,"[[0.020432742, -0.018192459, -0.06733605, -0.0..."
1,1,고집,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...,"[[0.011515088, -0.0400949, -0.24114151, 0.0015..."
2,2,신념,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...,"[[0.025614088, -0.020315323, -0.32384497, -0.0..."
3,3,주장,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...,"[[0.08533488, -0.1753439, -0.42259085, 0.06305..."
4,4,주관 뚜렷,당신은 뚜렷한 핵심신념(Core Belief)을 지니고 있으시군요. \n미국 정신과...,"[[-0.003093113, -0.08391347, -0.18019684, 0.06..."
...,...,...,...,...
257,257,보통 이상은 한다,어떠한 성취 상황에서 성공적으로 수행할 수 있다는 자신의 능력에 대한 신념을 자기효...,"[[0.017062016, -0.014257076, -0.163146, 0.0045..."
258,258,보통은 한다,어떠한 성취 상황에서 성공적으로 수행할 수 있다는 자신의 능력에 대한 신념을 자기효...,"[[0.031653333, -0.037678663, -0.1921397, 0.044..."
259,259,길 찾는 것,공간 사이의 관계를 파악할 수 있는 능력을 공간지각능력이라고 해요. 입체적인 물체나...,"[[-0.010229528, -0.037261523, -0.23678619, -0...."
260,260,다른 사람들을 편안하게 만드는거,당신은 친화성이 높은 것으로 보이네요. 심리학에서 개인의 성격을 설명하는 5요인 이...,"[[-0.00439104, -0.0044150553, -0.040940005, -0..."


In [31]:
def solution(text) :
    a = text
    ccc = []
    for i in df['vector'] :
        test = a
        toked = tok(test)
        input_ids = torch.tensor(vocab[toked]).unsqueeze(0)
        _, test_vector = bertmodel(input_ids = input_ids)
        cos = cosine_similarity(i, test_vector.detach().numpy()/len(toked) )
#         if cos > 1 :
#             cos = 0
        ccc.append(cos)
    print(max(ccc))
    return print(df['text'].iloc[ccc.index(max(ccc))])

In [33]:
solution('다른사람들을 편안하게 만드는거')

[[0.90117395]]
당신은 친화성이 높은 것으로 보이네요. 심리학에서 개인의 성격을 설명하는 5요인 이론에서는 친화성(Agreeableness)이 높은 사람이 대체로 다른 사람과 친밀하게 지내는 것을 선호하고, 공감을 잘 하는 것으로 설명하고 있어요. 
반면, 친화성이 낮은 경우에는 다른 사람과의 관계에 소극적인 편이고, 이기적인 경향을 나타낼 수 있어요. 
친화성이 높은 사람들은 심리적으로 건강한 생활을 할 수 있지만, 오히려 나 자신의 권리를 얻기 위해 싸우려는 자세가 부족할 수 있어요. 


In [34]:
solution('인내력')

[[1.0000001]]
긍정심리학(Positive Psychology)에 따르면 당신은 사람이 지닐 수 있는 긍정적인 6가지 덕목 중에서 절제(temperance)를 지니고 있는 사람이에요. 절제는 과도함으로부터 우리 자신을 보호하는 것과 관련이 있어요. 즉, 외부의 다른 사람의 도움이 없는 상황에서도 자신의 동기, 감정, 행동을 스스로 확인하고 다룰 수 있는 현실적인 능력이에요.
절제의 덕목에 해당하는 4가지 성격강점(Character Strength) 중에서도 당신의 강점은 자기통제력(Self-Regulation)이에요. 자신이 추구하는 가치를 이루기 위해서 자기 감정과 행동을 스스로 적절하게 통제할 수 있는 사람이지요. 자신의 욕구와 감정에 노예가 되어서 휘둘리는 것이 아니라 자신이 이들의 주인이 되어서 다룰 수 있는 사람이에요.
자기통제가 지나친 경우에는 지나치게 자신의 욕구를 억제하는 문제가 생길 수 있고, 자기통제가 존재하지 않는 경우에는 정리되지 않는 방종하는 삶을 살게 될 수 있어요. 균형을 이룬 자기통제는 당신이 바라는 일을 가장 효과적으로 달성할 때 무엇보다도 큰 도움이 되는 자산이에요.


In [26]:
solution('언젠가는 해낸다')

[[0.9187023]]
어떠한 성취 상황에서 성공적으로 수행할 수 있다는 자신의 능력에 대한 신념을 자기효능감이라고 해요. 자기효능감이 높은 사람들은 과제나 상황에 대한 자신감이 있어서 자신이 노력하면 좋은 성과를 얻을 수 있다고 믿는 경향이 있어요. 
자기효능감이 높은 사람은 자신의 삶을 스스로 통제하고 결정한다고 믿는 편이에요. 학습 또는 업무에 대한 동기가 높은 편이기 때문에 수행에 있어서도 지속적이고 적극적인 경향을 보여요. 이러한 경향은 성취 결과와도 관련이 있어요. 반면, 자기효능감이 낮은 사람들은 자신의 삶은 자신의 통제를 벗어나 있다고 생각하는 편이에요. 대인관계에서도 회피적인 행동을 하는 경향이 있어요. 

우리가 불안한 상황, 부담되고 억압된 상황에 처하면 정서적으로 각성된 상태가 되는데, 이러한 정서적 각성은 자기효능감에 부정적인 영향을 미칠 수 있어요. 정서적 각성의 원인을 자신의 무능력으로 생각하는 사람은 환경적인 원인으로 돌리는 사람보다 더 낮은 자기효능감을 갖게 되지요. 자기효능감이 낮은 경우에는 이와 같은 위협적인 상황에서 자신의 해결능력에 대해 부족하다고 생각하기 때문에 회피하는 행동을 많이 나타내요. 그래서 낮은 자기효능감은 우울과도 밀접한 관련이 있어요.

자기효능감을 높이려면 성공 경험을 계속 반복하는 과정이 가장 중요해요. 또한, 다른 사람의 성취 행동이나 결과를 관찰하는 대리 경험을 통해서도 '나도 할 수 있다'는 효능감을 가질 수 있지요. 뿐만 아니라, 주변 사람들의 격려와 칭찬도 자기효능감을 높이는 데 영향을 미칠 수 있어요.
